**NOTE: This notebook is written for the Google Colab platform, which provides free hardware acceleration. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook, using a local GPU.**

The notebook uses the flashtorch package and we also use the image of the toucan from the same repository.

In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install flashtorch skorch

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import matplotlib.pyplot as plt
import torchvision.models as models
from flashtorch.saliency import Backprop
from skorch import NeuralNetClassifier, NeuralNet
from torchvision import transforms
from PIL import Image
import numpy as np
import torch

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
!mkdir -p data
!wget -nv -nc -O data/imagenet_classes https://www.dropbox.com/s/ma25i7w3jpqex2a/imagenet_classes?dl=1
!wget -nv -nc -O data/toucan.jpg https://github.com/MisaOgura/flashtorch/raw/master/examples/images/toucan.jpg
!wget -nv -nc -O data/car.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/b/b2/DNK_ambulance_new_design.jpg/640px-DNK_ambulance_new_design.jpg
!wget -nv -nc -O data/raccoon_example.png https://www.dropbox.com/s/qca8pk3x7ouvoo3/raccoon_example.jpg?dl=1
!wget -nv -nc -O data/nails.jpg https://www.dropbox.com/s/at02ej24yd7h1qc/nails.jpg?dl=1
!wget -nv -nc -O data/screws.jpg https://www.dropbox.com/s/1rp38ie35xih0dn/screws.jpg?dl=1

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
with open("data/imagenet_classes", "r") as file:
    classes = [c[:-1] for c in file.readlines()]

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

def preproc_image(img, size=224):
    if not isinstance(img, Image.Image):
        img = F.to_pil_image(img)

    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]

    transform = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize(means, stds)
    ])

    tensor = transform(img).unsqueeze(0)
    tensor.requires_grad = True
    return tensor

def decode_proba(proba, top=5):
    proba = proba.ravel()
    ind = np.argsort(proba)
    
    for c in reversed(ind[-top:]):
        print("{}:\t{} ({})".format(
            np.array2string(proba[c], precision=5),
            classes[c], c))

# Visual Interpretation of Neural Networks

Neural nets with deep learning represent a powerful machine learning paradigm. However, they are not known for their interpretability. Nevertheless, there are a few techniques that can provide some insight into whether a neural net is doing what it should or not. For tabular data, predictions can of course be explained with methods such as LIME. But there is also a couple of good techniques that work for images and we are going to showcase one of them in this notebook.

The approach itself is simple. We use the backprop to compute the sensitivity of the prediction to the various input pixels and visualize the result, which is sometimes referred to as a saliency map. The saliency map, roughly speaking, highlights the pixels in proportion to their relevance to the prediction. This allows us to inspect whether the network is paying attention to the correct portions of the image: i.e. that it actually predicts label "plane" because it recognizes the plane and not because most of the pixels in the background are blue.

## Loading the Model

We will start by loading a pretrained model that we are going to be testing. We also wrap it in the skorch interface to make the prediction of labels easier.

In [ ]:
use_gpu = torch.cuda.is_available()
device = "cuda" if use_gpu else "cpu"
model = models.densenet161(pretrained=True)

net = NeuralNetClassifier(
    torch.nn.Sequential(
        model,
        torch.nn.Softmax(dim=-1)
    ),
    device=device,
)

net.initialize();

We create the ``Backprop`` explainer for our model.

In [ ]:
backprop = Backprop(model)

We load a few images, run them through our model and display the saliency maps. Note how the maps highlight the salient pixels: the images of the toucan and the ambulance are especially interesting. For the toucan, the beak is highlighted, which makes sense: that is what a human would pay attention to when trying to distinguish it from other birds. Similarly for the ambulance – most attention is on the patterns and the lights.

When recognizing screws, the network seems to be focusing onto the spiral pattern and when recognizing nails, the heads seem to be important as well as the body of the nail.

In [ ]:
img = Image.open('data/toucan.jpg')
img_preproc = preproc_image(img)
backprop.visualize(img_preproc, None, guided=True, use_gpu=use_gpu)
decode_proba(net.predict_proba(img_preproc))

In [ ]:
img = Image.open('data/car.jpg')
img_preproc = preproc_image(img)
backprop.visualize(img_preproc, None, guided=True, use_gpu=use_gpu)
decode_proba(net.predict_proba(img_preproc))

In [ ]:
img = Image.open('data/screws.jpg')
img_preproc = preproc_image(img)
backprop.visualize(img_preproc, None, guided=True, use_gpu=use_gpu)
decode_proba(net.predict_proba(img_preproc))

In [ ]:
img = Image.open('data/nails.jpg')
img_preproc = preproc_image(img)
backprop.visualize(img_preproc, None, guided=True, use_gpu=use_gpu)
decode_proba(net.predict_proba(img_preproc))

In [ ]:
img = Image.open('data/raccoon_example.png')
img_preproc = preproc_image(img)
backprop.visualize(img_preproc, None, guided=True, use_gpu=use_gpu)
decode_proba(net.predict_proba(img_preproc))

## Other Visualization Techniques

Naturally, there are many other visualization techniques. To get some inspiration, you may explore the [Lucid repository](https://github.com/tensorflow/lucid) from Google, which showcases some cool things: including visualizations of preimages of filters from intermediate layers and such.